<h1>Email recipient recommendation</h1>

<i>Thomas Boudou, Guillaume Richard, Antoine Simoulin</i>

<p style="text-align: justify">It was shown that at work, employees frequently forget to include one or more recipient(s) before sending a message. Conversely, it is common that some recipients of a given message were actually not intended to receive the message. To increase productivity and prevent information leakage, the needs for effective <b>email recipient recommendation</b> systems are thus pressing.

In this challenge, you are asked to develop such a system, which, given the content and the date of a message, recommends a list of <b>10 recipients ranked by decreasing order of relevance</b>.</p>

In [1]:
# Requirements
%matplotlib inline
import random
import pandas as pd
import numpy as np
# do not display warnings
import warnings
warnings.filterwarnings("ignore")

# Functions files are saved in "src/" directory.
import sys
sys.path.append('src/')
from accuracy_measure import *

In [2]:
from load_data import *

# load files
# Data are saved in "data/" directory
path_to_data = 'data/'
training, training_info, test, test_info, y_df = load_data(path_to_data)


# join train and test files
X_df = join_data(training_info, training)
X_sub_df = join_data(test_info, test)

# remove non authorise adress from y_df
y_df = clean(y_df)

In [3]:
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import time

cachedStopWords = stopwords.words("english")

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    stemmer = PorterStemmer()
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

class TFIDF():
    def __init__(self):
        self.token_dict = {}
        self.tfidf=TfidfVectorizer(tokenizer=None, stop_words='english')

    def fit(self, X):
        for i in range(X.shape[0]):
            text = X.body[i]
            lowers = text.lower()
            s=string.punctuation.replace('@','')
            s=s.replace('+','')
            no_punctuation = lowers.translate(str.maketrans('','',s))
            y = " ".join(no_punctuation.split())
            y = ' '.join([word for word in y.split() if word not in cachedStopWords])
            self.token_dict[i] = y

        self.tfidf.fit(self.token_dict.values())


    def fit_transform(self, X):
        start_time = time.time()
        for i in range(X.shape[0]):
            text = X.body[i]
            lowers = text.lower()
            s=string.punctuation.replace('@','')
            s=s.replace('+','')
            no_punctuation = lowers.translate(str.maketrans('','',s))
            y = " ".join(no_punctuation.split())
            y = ' '.join([word for word in y.split() if word not in cachedStopWords])
            self.token_dict[i] = y

        X_tfidf = self.tfidf.fit_transform(self.token_dict.values())

        print('performed Tf-Idf in %2i seconds.' % (time.time() - start_time))
        return X_tfidf

    def transform(self, Y):
        start_time = time.time()
        Y_dict={}
        for i in range(Y.shape[0]):
            text = Y.body[i]
            lowers = text.lower()
            s=string.punctuation.replace('@','')
            s=s.replace('+','')

            no_punctuation = lowers.translate(str.maketrans('','',s))
            y = " ".join(no_punctuation.split())
            y = ' '.join([word for word in y.split() if word not in cachedStopWords])

            Y_dict[i] = y
        Y_tf_idf=self.tfidf.transform(Y_dict.values())

        print('performed Tf-Idf in %2i seconds.' % (time.time() - start_time))
        return Y_tf_idf


In [4]:
# transform each mail body into tfidf vector
# /!\ function can take 1-2 min to execute
TFIDF = TFIDF()
X_TFIDF = TFIDF.fit_transform(X_df) # resulting shape : (43613, 275988) / without tokenizer: (43613, 294517)
X_sub_TFIDF = TFIDF.transform(X_sub_df)

KeyboardInterrupt: 

In [7]:
X_TFIDF.shape

(43613, 294517)

<h2>Predictors</h2>

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def complete_prediction(k, sender, address_books, res_temp, K=10):
    # k the number of recipients to predict
    k_most = [elt[0] for elt in address_books[sender][:K] if elt[0] not in res_temp]
    k_most = k_most[:k]
    if len(k_most) < k: # sender n'a pas assez de contacts
        k_most.extend([0] * (k-len(k_most)))
    return k_most

<h3>Predictor closest message with Tf-Idf</h3>

In [9]:
class Predictor_TFIDF():
    def __init__(self, X, y, sender, address_books,N=10):
        self.train = X
        self.predict = y.values
        self.sender = sender
        self.address_books = address_books
        self.N = min(N,10)
        
    def prediction(self, X):
        res = []
        for i in range(X.shape[0]):
            cos = cosine_similarity(X[i],self.train).argsort()[:,0][0] # mail le plus proche
            if self.N != 0:
                res_temp = [self.predict[cos][0][:self.N]] # add the N first recipients of the closest e-mail
            else:
                 res_temp = []
            # if less than 10 recipients, complete the prediction with more frequents users
            if len(res_temp) < 10:
                res_temp.extend(complete_prediction(10-len(res_temp),self.sender, self.address_books, res_temp))
            res.append(res_temp)
        return res

<h3>Predictor KNN</h3>

In [10]:
class Predictor_KNN():
    def __init__(self, X, y, sender, address_books,N=10):
        self.train = X
        self.predict = y.values
        self.sender = sender
        self.N = min(N,10)
        self.address_books = address_books

    def prediction(self, X):
        res = []
        for i in range(X.shape[0]):
            cos = cosine_similarity(X[i],self.train).argsort()[:,:30][0] # 30 mails les plus proches
            if self.N != 0:
                NN_recpt = {}
                for j in range(30):
                    for k in range(len(self.predict[cos[j]])):
                        if self.predict[cos[j]][k] in NN_recpt:
                            NN_recpt[self.predict[cos[j]][k]]+= 1
                        else:
                            NN_recpt[self.predict[cos[j]][k]] = 1
                res_temp = [k for k in sorted(NN_recpt, key=NN_recpt.get, reverse=True)][:10]
                #list(dict(sorted(NN_recpt.items(), key=operator.itemgetter(1), reverse=True)[:10]))
            else:
                 res_temp = []
            # if less than 10 recipients, complete the prediction with more frequents users
            if len(res_temp) < 10:
                res_temp.extend(complete_prediction(10-len(res_temp),self.sender, self.address_books, res_temp))
            res.append(res_temp)
        return res

<h3>Predictor Tf-Idf centralized centroïd</h3>

In [11]:
import scipy
from sklearn.preprocessing import normalize


class Predictor_CTFIDF():
    def __init__(self, X, y, sender, address_books,N=10, NMost=30):
        self.N = min(N,10)
        self.train = X
        self.predict = y.values
        self.sender = sender
        self.address_books = address_books
        self.X_recpt = {}
        
        # perform centroid Tf-Idf. i.e each 10 most frequent recipients is represented 
        # by an average of all mail he received.
        # exctract 10 most frequents recipients
        self.k_most = [elt[0] for elt in address_books[sender][:NMost]]
        # perform average Tf-Idf on 10 most frequents recipients
        for recpt in self.k_most: # loop trough 10 most frequents recipients
            for i in range(X.shape[0]): # loop trough all mails send by sender
                if recpt in self.predict[i]: # if recipients is in mail
                    if recpt in self.X_recpt:
                        self.X_recpt[recpt] += X[i,:]
                    else:
                        self.X_recpt[recpt] = X[i,:]
            #self.X_recpt[recpt] = normalize(self.X_recpt[recpt], norm='l2', axis=1) # normalize tfidf vector

    def prediction(self, X):
        res = []
        cos = {}
        for i in range(X.shape[0]):
            # cosine similarity with 10 most frequents recpt
            for recpt, value in self.X_recpt.items():
                #if isinstance(self.X_recpt[recpt], list): # check that sender has more than 10 recipients
                cos[recpt] = cosine_similarity(X[i],self.X_recpt[recpt])#.argsort()[:,0][0]
                #else:
                #    cos[recpt] = 1
            #if i==0:
            #    print(cos)
            #if 'no.address@enron.com' in cos:
            #    cos['no.address@enron.com'] += 100
            if self.N != 0:
                # return the 10 most frequent recipients in order 
                # given by similarity to their centroid Tf-Idf representation
                res_temp = [k for k in sorted(cos, key=cos.get, reverse=True)][:10]
                #list(dict(sorted(cos.items(), key=operator.itemgetter(1), reverse=False)[:10]))
            else:
                 res_temp = []
            # if less than 10 recipients, complete the prediction with more frequents users
            if len(res_temp) < 10:
                res_temp.extend(complete_prediction(10-len(res_temp),self.sender, self.address_books, res_temp))
            res.append(res_temp)
        return res

<h2>Cross validation module</h2>

In [14]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold
# splitting data for cross validation
skf = KFold(n_splits=10, shuffle=True, random_state=None)
#print('%10s | %40s | %10s | %10s' %('sender_nb', 'sender', 'accuracy','accuracy freq'))
#print('%10s + %40s + %10s + %13s' %(10*'-', 40*'-', 10*'-', 13*'-'))

print('K-Fold')
print('------------')
final_error=0
for train_is, test_is in skf.split(y_df):
    X_tfidf_train = X_TFIDF[train_is].copy()
    y_train = y_df.recipients.loc[train_is].copy()
    X_tfidf_test = X_TFIDF[test_is].copy()
    y_test = y_df.recipients.loc[test_is].copy()
    X_test_df = X_df.loc[test_is].copy()
    X_train_df = X_df.loc[train_is].copy()
    
    i=0
    pdt = {}
    pdt_freq = {}
    accuracy = {}
    accuracy_freq = {}
    accuracy_TOT = 0
    sender_test = X_test_df.sender.unique().tolist()
    y_pred = np.empty((X_test_df.shape[0],10),dtype=object)
    y_pred_freq = np.empty((X_test_df.shape[0],10),dtype=object)

    for sender in sender_test:
        #print('%10s | %40s | ' %(sender_test.index(sender), sender), end='')
        # indices corresponding to the sender
        sender_train_is = np.array(X_train_df.sender == sender)
        sender_test_is = np.array(X_test_df.sender == sender)
        
        pdt[sender] = Predictor_CTFIDF(X_tfidf_train[sender_train_is], y_train[sender_train_is], sender, address_books)
        pdt_freq[sender] = Predictor_CTFIDF(X_tfidf_train[sender_train_is], y_train[sender_train_is], sender, address_books, N=0)
        y_pred[sender_test_is] = pdt[sender].prediction(X_tfidf_test[sender_test_is])
        y_pred_freq[sender_test_is] = pdt_freq[sender].prediction(X_tfidf_test[sender_test_is])
        
        accuracy[sender] = mapk(y_test[sender_test_is], y_pred[sender_test_is])
        accuracy_freq[sender] = mapk(y_test[sender_test_is], y_pred_freq[sender_test_is])
        
        accuracy_TOT += accuracy[sender]
        #print('%10s | %13s' %(round(accuracy[sender],2),round(accuracy_freq[sender],2)))
        

    print('%30s'%(90*'-'))
    print('error TOT = %.2f' %(accuracy_TOT/len(accuracy)))
    final_error=final_error+accuracy_TOT/len(accuracy)
final_error=final_error/10

K-Fold
------------
------------------------------------------------------------------------------------------
error TOT = 0.53
------------------------------------------------------------------------------------------
error TOT = 0.53
------------------------------------------------------------------------------------------
error TOT = 0.53
------------------------------------------------------------------------------------------
error TOT = 0.52
------------------------------------------------------------------------------------------
error TOT = 0.53


KeyboardInterrupt: 

In [104]:
# print(y_pred[np.array(X_test_df.sender == 'kimberly.hillis@enron.com')])

In [105]:
# print(y_pred_freq[np.array(X_test_df.sender == 'kimberly.hillis@enron.com')])

In [106]:
# print(y_test[np.array(X_test_df.sender == 'kimberly.hillis@enron.com')].values)

<h2>Create submission</h2>

In [8]:
def create_submission(y_pred,X_test_df):

    predictions_towrite={}
    x_test=X_test_df.values
    for i in range(len(y_pred)):
        recipients=y_pred[i]
        mid=x_test[i][0]
        predictions_towrite[mid]=recipients

    count=0
    with open('./pred_custom.txt', 'w') as my_file:
        my_file.write('mid,recipients' + '\n')
        for ids, preds in predictions_towrite.items():
            count=count+1
            r=str(ids)+","
            for s in preds:
                r=r+" "+str(s)
            r=r+'\n'
            my_file.write(r)

In [15]:
sender_test = X_sub_df.sender.unique().tolist()
y_pred = np.empty((X_sub_df.shape[0],10),dtype=object)
count=0
pdt = {}

for sender in sender_test:
    # indices corresponding to the sender
    sender_train_is = np.array(X_df.sender == sender)
    sender_test_is = np.array(X_sub_df.sender == sender)
    
    y_train_all = y_df.recipients.copy()
    pdt[sender] = Predictor_CTFIDF(X_TFIDF[sender_train_is], y_train_all[sender_train_is], sender, address_books)
    y_pred[sender_test_is] = pdt[sender].prediction(X_sub_TFIDF[sender_test_is])
    
create_submission(y_pred,X_sub_df)